In [8]:
!pip install SpeechRecognition pydub gTTS


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [9]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
from google.colab import drive,files
import speech_recognition as sr
from pydub import AudioSegment
import io
from gtts import gTTS
from IPython.display import Audio
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
drive.mount('/content/drive')

In [ ]:
model_path = '/content/drive/MyDrive/dialogpt_model'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Model Response Function
def get_response(input_text):
    encoded_input = tokenizer(
        input_text + tokenizer.eos_token,
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

    response = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response




In [17]:
# Upload Audio and Recognize
uploaded = files.upload()
file_name = next(iter(uploaded))

if not file_name.endswith(".wav"):
    audio = AudioSegment.from_file(io.BytesIO(uploaded[file_name]))
    file_name = file_name.rsplit(".", 1)[0] + ".wav"
    audio.export(file_name, format="wav")

recognizer = sr.Recognizer()
with sr.AudioFile(file_name) as source:
    audio_data = recognizer.record(source)
    try:
        user_input = recognizer.recognize_google(audio_data)
        print("🗣️ Recognized Speech:", user_input)
    except sr.UnknownValueError:
        print("Could not understand the audio.")
        user_input = ""

Saving New Recording 5.mp3 to New Recording 5.mp3
🗣️ Recognized Speech: how do you feel today


In [18]:
# Get Bot Response
bot_reply = get_response(user_input)
print("Model Input:", user_input)
print("Model Output:", bot_reply)
tts = gTTS(bot_reply)
tts.save("reply.mp3")
Audio("reply.mp3", autoplay=True)

Model Input: how do you feel today
Model Output: I feel a little lightheaded. What's up?
